In [ ]:
import csv
from numpy import genfromtxt
import numpy as np
import pandas as pd
from random import random
import math
import sklearn.linear_model
import sys
import torch.autograd
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

# Import relevant files
filename1 = 'train_set_attr_scld.csv'
train_set_attr_scld = genfromtxt(filename2, delimiter=',')

filename2 = 'test_set_scld.csv'
test_set_scld = genfromtxt(filename3, delimiter=',')

# ExperimentsCNN.txt contains parameters for the neural network as follows:
# experiment num, iter, lrate, ovrsmpl, epochnum, ftr_size, ftr_1, ftr_2 ... ftr_n

filename = 'experimentsCNN.txt'
lineNum = 48
countIter = -1
# Read input file to run processes in parallel on the cluster
with open(filename) as f:
    for line in f:
        lineNum = lineNum + 1
        if(lineNum == int(sys.argv[1])):
            entries = line.split(",")
            iterations = int(entries[1])
            lrate = float(entries[2])
            ovrsmpl = int(entries[3])
            epochNum = int(entries[4])
            ftr_size = int(entries[5])
            ftrs = list()
            for feature in range(ftr_size):
                ftrs.append(int(entries[6 + feature]))
    
output = np.empty((10,3))

# Loop through every iteration/trial 
for iteration in (range(iterations)):
    feature_size = ftr_size
    
#   Define network architecture
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()  
            self.conv1 = nn.Conv1d(ftr_size, 45, 1) 
            self.conv2 = nn.Conv1d(45, 15, 1)
            self.fc1 = nn.Linear(15, 120) 
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 3)     

        def forward(self, x):
            x =x.unsqueeze(dim = 2)
            x = self.conv1(x)
            x = F.relu(x)
            x = F.max_pool1d(x, 1)
            x = self.conv2(x)
            x = F.relu(x)
            x = F.max_pool1d(x, 1)
            x = x.view(-1, self.num_flat_features(x))
            x = self.fc1(x)
            x = F.relu(x)
            x = self.fc2(x)
            x = F.relu(x)
            x = self.fc3(x)  
            x = F.log_softmax(x, dim = 1)
            return x
        
        def num_flat_features(self, x):
            size = x.size()[1:]
            num_features = 1
            for s in size:
                num_features *= s
            return num_features

#   Create instance of network architecture
    net = Net()
    
#  Set up optimizer (stochastic gradient descent) and loss criterion (cross entropy)
    learning_rate = lrate
    p = 0.9
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=p)
    criterion = nn.CrossEntropyLoss()
    
    #  Creating the training and testing set loaders
    import torch.utils.data 
    batch_size = 100

#   Dataset subclass: EphysDataset that returns each row of the array as a PyTorch Tensor
    class EphysDataset(torch.utils.data.Dataset):
        def __init__(self, numpy_arr):
            self.numpy_arr = numpy_arr

        def __len__(self):
            return len(self.numpy_arr)

        def __getitem__(self, index):
            arr = self.numpy_arr[index,:]
            sample = torch.from_numpy(arr)
            sample = sample.type(torch.FloatTensor)
            return arr

#   Convert NumPy array to PyTorch Dataset
    train_set_attr_dtst = EphysDataset(train_set_attr_scld)
    test_set_dtst = EphysDataset(test_set_scld)

#  Load Datasets with DataLoader        
    train_loader_attr = torch.utils.data.DataLoader(train_set_attr_dtst, batch_size = batch_size, shuffle = True)
    test_loader = torch.utils.data.DataLoader(test_set_dtst, batch_size = batch_size, shuffle = True)
    
    inputDataset = train_loader_attr
        
    # Sample 2D array (eg. for one cell type)
    # [unit1: [fA nA pA wTP wPT isi reg b wvfrm(29)]
    #  unit2: [fA nA pA wTP wPT isi reg b wvfrm(29)]
    # ..........................
    #  unitN: [fA nA pA wTP wPT isi reg b wvfrm(29)]]
    
#   Training the network
    epochs = epochNum
    log_interval= 1000
    
    for epoch in range(epochs): 
        
        for batch_idx, arr in enumerate(inputDataset):
            
#           Extract the data and target separately and make them PyTorch Tensors
            data = arr[:,ftrs]
            target = arr[:,0] - 1
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            optimizer.zero_grad()
            
#           Pass data to NN and calculate loss
            net_out = net(data)
            loss = criterion(net_out, target)
        
#           Backpropogation and gradient descent
            loss.backward()
            optimizer.step()

#   Testing the network
    test_loss = 0
    correct = 0
    
    for arr in test_loader:
#       Create data and target tensors
        idx = -1;
        data = arr[:,ftrs]
        target = arr[:,0] - 1
        target = target.numpy() 
        for num in target:
            idx = idx + 1
            if num == -1:
                target[idx] = None
        target = torch.from_numpy(target)
        target = target.type(torch.LongTensor)
        data = data.view(-1, feature_size)
        data = data.type(torch.FloatTensor)
        data, target = torch.autograd.Variable(data), torch.autograd.Variable(target)
#       Test the network on validation data
        net_out = net(data)  
    
#       Sum up batch loss
        loss = criterion(net_out, target)
        test_loss += loss.item()
        
#       Get the index of the max log-probability
        pred = net_out.data.max(1)[1]  
        correct += pred.eq(target.data).sum()

    countIter += 1
    output[countIter, 0] = test_loss
    output[countIter, 1] = correct
    output[countIter, 2] = 100. * correct / len(test_loader.dataset)

np.savetxt('output' + str(sys.argv[1]) + '.csv', output, delimiter = ",")